In [1]:
import json
from multiprocessing import Pool

import fair
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open('../data_input/fair-1.6.2-ar6/fair-1.6.2-wg3-params.json') as f:
    config_list = json.load(f)

In [3]:
df = pd.read_csv('../data_input/ar6/gmst-assessment.csv')
ar6_gmst = df['temperature'].values

In [4]:
emissions_in = {}
results_out = {}

## convenience function for running FaIR config with each emission species

In [5]:
def run_fair(args):
    thisC, thisF, thisT, _, thisOHU, _, thisAF = fair.forward.fair_scm(**args)
    return (thisC[:,0], thisT, np.sum(thisF[:,:43], axis=1), np.sum(thisF[:,35:41], axis=1), np.sum(thisF, axis=1))

def fair_process(emissions):
    updated_config = []
    for i, cfg in enumerate(config_list):
        updated_config.append({})
        for key, value in cfg.items():
            if isinstance(value, list):
                updated_config[i][key] = np.asarray(value)
            else:
                updated_config[i][key] = value
        updated_config[i]['emissions'] = emissions
        updated_config[i]['diagnostics'] = 'AR6'
        updated_config[i]["efficacy"] = np.ones(45)
        updated_config[i]["gir_carbon_cycle"] = True
        updated_config[i]["temperature_function"] = "Geoffroy"
        updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
        updated_config[i]["fixPre1850RCP"] = False
    #    updated_config[i]["scale"][43] = 0.6
        updated_config[i]["F_solar"][270:] = 0
    
    if __name__ == '__main__':
        with Pool(3) as pool:
            result = list(tqdm(pool.imap(run_fair, updated_config), total=len(updated_config)))
    
    result_t = np.array(result).transpose(1,2,0)
    c, t, f_ant, f_aer, f_tot = result_t
    temp_rebase = t - t[100:151,:].mean(axis=0)
    
    return c, temp_rebase, f_ant, f_aer, f_tot

In [ ]:
updated_config = []
for i, cfg in enumerate(config_list):
    updated_config.append({})
    for key, value in cfg.items():
        if isinstance(value, list):
            updated_config[i][key] = np.asarray(value)
        else:
            updated_config[i][key] = value
    updated_config[i]['emissions'] = emissions_out[scenario]
    updated_config[i]['diagnostics'] = 'AR6'
    updated_config[i]["efficacy"] = np.ones(45)
    updated_config[i]["gir_carbon_cycle"] = True
    updated_config[i]["temperature_function"] = "Geoffroy"
    updated_config[i]["aerosol_forcing"] = "aerocom+ghan2"
    updated_config[i]["fixPre1850RCP"] = False
#    updated_config[i]["scale"][43] = 0.6
    updated_config[i]["F_solar"][270:] = 0
    
results_out[scenario] = {}
c, t, f_ant, f_aer, f_tot = result_t
temp_rebase = t - t[100:151,:].mean(axis=0)
results_out[scenario]['co2_conc'] = c
results_out[scenario]['temp'] = temp_rebase
results_out[scenario]['F_ant'] = f_ant
results_out[scenario]['F_aer'] = f_aer
results_out[scenario]['F_tot'] = f_tot

In [ ]:
pl.fill_between(np.arange(1750.5,2111), np.percentile(results_out[scenario]['temp'], 5, axis=1), np.percentile(results_out[scenario]['temp'], 95, axis=1))
pl.plot(np.arange(1750.5,2111), np.percentile(results_out[scenario]['temp'], 50, axis=1), color='k')
pl.plot(np.arange(1850.5, 2021), ar6_gmst, color='r')

pl.grid()

In [ ]:
pl.plot(np.arange(1750.5,2111), np.percentile(results_out['ssp245']['temp'], 50, axis=1), color='k')
pl.plot(np.arange(1750.5,2111), np.percentile(results_out['ch4_50']['temp'], 50, axis=1), color='r')
pl.xlim(2015, 2050)
pl.ylim(1.1, 1.9)
pl.grid()